In [ ]:
#Installations
!pip install accelerate

In [ ]:
!pip install langchain

In [ ]:
#importing necessary libraries
from transformers import AutoTokenizer, pipeline
import transformers
import torch
import accelerate
from langchain import HuggingFacePipeline
from transformers import AutoTokenizer, pipeline
import torch
from langchain import PromptTemplate,  LLMChain
import re


In [ ]:
#download the model
model = "tiiuae/falcon-7b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model)


In [ ]:
#defining the pipeline
pipeline = pipeline(
    task="text-generation", #task
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
    max_length=1000,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [ ]:
#defining the LLM chain
llm = HuggingFacePipeline(pipeline = pipeline, model_kwargs = {'temperature':0})
template = """
identify attack id from the report_str string, Access the following link: https://attack.mitre.org/techniques/enterprise/ and based on the data,
and explain why. Dont limit your output, you can use as many words as you like
Report: {report_str}
Answer:"""
prompt = PromptTemplate(template=template, input_variables=["report_str"])

llm_chain = LLMChain(prompt=prompt, llm=llm)

In [ ]:
report_str = "The malware\u2019s capabilities include taking a system survey, access to the filesystem, executing commands and a reverse shell"

In [ ]:
try:
    print(llm_chain.run(report_str))  # Assuming it expects a string directly
except TypeError as e:
    print(f"Error: {e}")

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


 

Based on the information provided the attack type could be a phishing attack. The attack string'malware is a type of software that can be used to gain unauthorized access to a person or system without the user's knowledge' is indicative of a phishing attack as it is designed to gain access to a system through tricking the user into downloading malicious software. Additionally, the attack string mentions accessing a system survey, access to the file system, and executing commands, which are all possible ways for a phishing attacker to gain unauthorized access without the user's consent. 


In [ ]:
len(llm_chain.run(report_str))

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


277